In [0]:
# import libraries
import pyspark.sql.functions as F
from pyspark.sql.types import *
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from pyspark.sql import functions as f
from pyspark.sql import SQLContext
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import isnan, when, count, col, isnull, percent_rank, avg, mean
from pyspark.sql.functions import min
from pyspark.sql.functions import col, max
from pyspark.sql.functions import format_string
from pyspark.sql.functions import substring
from pyspark.sql.functions import concat_ws
from pyspark.sql.functions import concat
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import lit
from pyspark.sql.functions import to_utc_timestamp
from pyspark.sql.functions import expr
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import instr
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType

from pyspark.ml.linalg import DenseVector, SparseVector, Vectors
from pyspark.ml.feature import VectorAssembler, StandardScaler, StringIndexer,OneHotEncoder
from pyspark.ml.classification import MultilayerPerceptronClassifier


from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.classification import GBTClassifier

from pyspark.ml.classification import RandomForestClassifier, DecisionTreeClassifier, LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

In [0]:
#Initializes blob storage credentials/location
blob_container = "w261-sec4-group2" # The name of your container created in https://portal.azure.com
storage_account = "kdevery" # The name of your Storage account created in https://portal.azure.com
secret_scope = "sec4-group2" # The name of the scope created in your local computer using the Databricks CLI
secret_key = "w261-key" # The name of the secret key created in your local computer using the Databricks CLI 
blob_url = f"wasbs://{blob_container}@{storage_account}.blob.core.windows.net"
mount_path = "/mnt/mids-w261"

#Points to SAS token
spark.conf.set(
  f"fs.azure.sas.{blob_container}.{storage_account}.blob.core.windows.net",
  dbutils.secrets.get(scope = secret_scope, key = secret_key)
)

In [0]:
display(dbutils.fs.ls(f"{blob_url}"))

path,name,size,modificationTime
wasbs://w261-sec4-group2@kdevery.blob.core.windows.net/MLPC_predictions_df/,MLPC_predictions_df/,0,1670111625000
wasbs://w261-sec4-group2@kdevery.blob.core.windows.net/MLPC_predictions_df_no_undersampling/,MLPC_predictions_df_no_undersampling/,0,1670192176000
wasbs://w261-sec4-group2@kdevery.blob.core.windows.net/feature_engineered_data/,feature_engineered_data/,0,1668924639000
wasbs://w261-sec4-group2@kdevery.blob.core.windows.net/feature_engineered_data_test/,feature_engineered_data_test/,0,1668924670000
wasbs://w261-sec4-group2@kdevery.blob.core.windows.net/feature_engineered_train_data/,feature_engineered_train_data/,0,1668559613000
wasbs://w261-sec4-group2@kdevery.blob.core.windows.net/merged_cleaned_data/,merged_cleaned_data/,0,1669494945000
wasbs://w261-sec4-group2@kdevery.blob.core.windows.net/merged_cleaned_data_test/,merged_cleaned_data_test/,0,1669495012000
wasbs://w261-sec4-group2@kdevery.blob.core.windows.net/merged_cleaned_data_train/,merged_cleaned_data_train/,0,1669495000000
wasbs://w261-sec4-group2@kdevery.blob.core.windows.net/merged_data/,merged_data/,0,1669494746000
wasbs://w261-sec4-group2@kdevery.blob.core.windows.net/number_flights_and_delay_rate/,number_flights_and_delay_rate/,0,1669961275000


In [0]:
#Read processed folds from the blob
d_processed = {}

d_processed['df1'] = spark.read.parquet(f"{blob_url}/processed_fold_1")
d_processed['df2'] = spark.read.parquet(f"{blob_url}/processed_fold_2")
d_processed['df3'] = spark.read.parquet(f"{blob_url}/processed_fold_3")
d_processed['df4'] = spark.read.parquet(f"{blob_url}/processed_fold_4")
d_processed['df5'] = spark.read.parquet(f"{blob_url}/processed_fold_5")

In [0]:
def undersample(data, verbose = False):
    """
    Under samples the majority class
    """

    delay_count = data.filter(f.col('label') == 1 ).count()
    non_delay_count = data.filter(f.col('label') == 0 ).count()
    #   print(f' total count : {data.count()}')
    #   print(f' delayed count : {delay_count}')
    #   print(f' non delayed count : {non_delay_count}')

    fraction_undersample = delay_count / non_delay_count
    #   print(f' delayed / non delayed: {fraction_undersample}')

    train_delayed = data.filter(f.col('label') == 1)
    #   print(f' non delayed count df : {train_delayed.count()}')

    train_non_delay_undersample = data.filter(f.col('label') == 0).sample(withReplacement=False, fraction=fraction_undersample, seed = 42)
    #   print(f' oversampled delayed count : {train_non_delay_undersample.count()}')

    data_undersampled = train_delayed.union(train_non_delay_undersample)
    #   print(f' train_df Oversampled : {train_undersampled.count()}')

    return data_undersampled

In [0]:
d_processed_undersampled = {}

d_processed_undersampled['df1'] = undersample(d_processed['df1'])
d_processed_undersampled['df2'] = undersample(d_processed['df2'])
d_processed_undersampled['df3'] = undersample(d_processed['df3'])
d_processed_undersampled['df4'] = undersample(d_processed['df4'])
d_processed_undersampled['df5'] = undersample(d_processed['df5'])

In [0]:
d_processed['df1'].groupBy('label').count().show()

+-----+-------+
|label|  count|
+-----+-------+
|  0.0|5802362|
|  1.0|1270968|
+-----+-------+



In [0]:
d_processed_undersampled['df1'].groupBy('label').count().show()

+-----+-------+
|label|  count|
+-----+-------+
|  1.0|1270968|
|  0.0|1271834|
+-----+-------+



### Model Building

In [0]:
%run "/Shared/w261_Section4_Group2/Phase 3/custom_cv_module"

#### Gradient Boosting Classifier - Cross Validation

In [0]:
gbt_model = GBTClassifier(labelCol="label", featuresCol="scaled_feature_vector")

grid = ParamGridBuilder()\
            .addGrid(gbt_model.maxDepth, [5,10])\
            .addGrid(gbt_model.minInfoGain,[0.0,0.2,0.4])\
            .addGrid(gbt_model.maxBins, [32,64])\
            .build()
            
# Example using F0.5 score for evaluator
evaluator = MulticlassClassificationEvaluator(metricName='fMeasureByLabel', beta=0.5, metricLabel=1)

In [0]:
# run cross validation & return the crossvalidation F0.5 score for 'test' set
cv = CustomCrossValidator(estimator=gbt_model, estimatorParamMaps=grid, evaluator=evaluator,splitWord =('train','val'), cvCol = 'cv',parallelism=10)

In [0]:
cvModel = cv.fit(d_processed_undersampled)

fold 1 start...
fold 1 end
fold 2 start...
fold 2 end
fold 3 start...
fold 3 end
fold 4 start...
fold 4 end
fold 5 start...
fold 5 end
Best Model:  {Param(parent='GBTClassifier_19d198aace14', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30].'): 5, Param(parent='GBTClassifier_19d198aace14', name='minInfoGain', doc='Minimum information gain for a split to be considered at a tree node.'): 0.0, Param(parent='GBTClassifier_19d198aace14', name='maxBins', doc='Max number of bins for discretizing continuous features.  Must be >=2 and >= number of categories for any categorical feature.'): 32} Detailed Score [0.7304984654682206, 0.7229489628384095, 0.7354924460440803, 0.7274454450893229, 0.694943946700294] Avg Score 0.7222658532280655


In [0]:
# make predictions
predictions = cvModel.transform(d_processed_undersampled['df1'])

display(predictions.groupby('label', 'prediction').count())

label,prediction,count
1.0,1.0,1128344
1.0,0.0,142624
0.0,1.0,841879
0.0,0.0,429955


In [0]:
fbeta = cvModel.avgMetrics[0]
print(f"Gradient Boosting F0.5 Score: {fbeta}")

Gradient Boosting F0.5 Score: 0.7222658532280655


#### Full Train and Test

In [0]:
# Read in processed training and test data

processed_train_df = spark.read.parquet(f"{blob_url}/processed_train")
processed_test_df = spark.read.parquet(f"{blob_url}/processed_test")

In [0]:
processed_train_undersampled_df = undersample(processed_train_df)

In [0]:
final_gbt_model = GBTClassifier(labelCol="label", featuresCol="scaled_feature_vector",maxDepth=5, minInfoGain=0, maxBins=64)
evaluator = MulticlassClassificationEvaluator(metricName='fMeasureByLabel', beta=0.5, metricLabel=1)
gbtModel = final_gbt_model.fit(processed_train_undersampled_df)

In [0]:
predictions = gbtModel.transform(processed_test_df)

In [0]:
display(predictions.groupby('label', 'prediction').count())

label,prediction,count
1.0,1.0,462014
0.0,1.0,319358
1.0,0.0,556689
0.0,0.0,4521245


In [0]:
evaluator.evaluate(predictions)

Out[20]: 0.5574236322601926

In [0]:
display(predictions)

label,scaled_feature_vector,index,rawPrediction,probability,prediction
0.0,"Map(vectorType -> sparse, length -> 39, indices -> List(0, 1, 2, 3, 4, 9, 22, 30, 31, 33, 35, 36, 37, 38), values -> List(0.6062867010278674, 0.4552178936348829, 1.9962981366261658, 1.1771165614071244, 1.1807240188784864, 2.001448286999518, 4.804541102597839, 0.5290014255667469, 2.5358333967474525, 0.3529775526800816, 2.2518971367325515, 0.2623600525700863, 2.453487744696793, 0.4321511055947388))",0,"Map(vectorType -> dense, length -> 2, values -> List(0.7404091396473503, -0.7404091396473503))","Map(vectorType -> dense, length -> 2, values -> List(0.8146961451462541, 0.1853038548537459))",0.0
0.0,"Map(vectorType -> sparse, length -> 39, indices -> List(0, 1, 2, 3, 4, 9, 21, 30, 31, 33, 35, 36, 37, 38), values -> List(0.6062867010278674, 0.4552178936348829, 1.9962981366261658, 0.8181980723075531, 1.518073738558054, 2.001448286999518, 5.254269120180973, 0.34977829198919363, 0.9689112552858259, 0.433041426600483, 2.3371183222456193, 0.27964920300085777, 2.453487744696793, 0.4321511055947388))",1,"Map(vectorType -> dense, length -> 2, values -> List(0.7575323699111516, -0.7575323699111516))","Map(vectorType -> dense, length -> 2, values -> List(0.8198105884433862, 0.18018941155661383))",0.0
0.0,"Map(vectorType -> sparse, length -> 39, indices -> List(0, 1, 2, 3, 4, 9, 21, 30, 31, 33, 35, 36, 37, 38), values -> List(0.6062867010278674, 0.4552178936348829, 1.9962981366261658, 0.8181980723075531, 1.1807240188784864, 2.001448286999518, 5.254269120180973, 0.4050167164495406, 0.9689112552858259, 0.433041426600483, 2.3371183222456193, 0.27964920300085777, 2.453487744696793, 0.4321511055947388))",2,"Map(vectorType -> dense, length -> 2, values -> List(0.7678089326639495, -0.7678089326639495))","Map(vectorType -> dense, length -> 2, values -> List(0.8228267885663978, 0.1771732114336022))",0.0
0.0,"Map(vectorType -> sparse, length -> 39, indices -> List(0, 1, 2, 3, 9, 21, 30, 31, 33, 35, 36, 37, 38), values -> List(0.6062867010278674, 0.4552178936348829, 1.9962981366261658, 0.8181980723075531, 2.001448286999518, 5.254269120180973, 0.34688490538266537, 0.9689112552858259, 0.433041426600483, 2.3371183222456193, 0.27964920300085777, 2.453487744696793, 0.4321511055947388))",3,"Map(vectorType -> dense, length -> 2, values -> List(0.756370994033921, -0.756370994033921))","Map(vectorType -> dense, length -> 2, values -> List(0.8194672138777144, 0.18053278612228563))",0.0
0.0,"Map(vectorType -> sparse, length -> 39, indices -> List(0, 1, 2, 3, 4, 9, 22, 30, 31, 33, 35, 36, 37, 38), values -> List(0.6062867010278674, 0.4552178936348829, 1.9962981366261658, 0.722940084679231, 2.361448037756973, 2.001448286999518, 4.804541102597839, 0.38154845811321203, 0.7047198828462189, 0.5403663049134857, 1.8916360906620093, 0.4360534659923761, 2.453487744696793, 0.4321511055947388))",4,"Map(vectorType -> dense, length -> 2, values -> List(0.7903396591115136, -0.7903396591115136))","Map(vectorType -> dense, length -> 2, values -> List(0.8293007044890806, 0.17069929551091945))",0.0
0.0,"Map(vectorType -> sparse, length -> 39, indices -> List(0, 1, 2, 3, 4, 9, 22, 30, 31, 33, 35, 36, 37, 38), values -> List(0.6062867010278674, 0.4552178936348829, 1.9962981366261658, 0.722940084679231, 0.8433742991989189, 2.001448286999518, 4.804541102597839, 0.3154302374467945, 0.7047198828462189, 0.5403663049134857, 1.8916360906620093, 0.4360534659923761, 2.453487744696793, 0.4321511055947388))",5,"Map(vectorType -> dense, length -> 2, values -> List(0.7777647806708824, -0.7777647806708824))","Map(vectorType -> dense, length -> 2, values -> List(0.825710937963787, 0.17428906203621297))",0.0
1.0,"Map(vectorType -> sparse, length -> 39, indices -> List(0, 1, 2, 3, 4, 7, 9, 22, 30, 31, 33, 35, 36, 37, 38), values -> List(0.6062867010278674, 0.4552178936348829, 1.9962981366261658, 0.722940084679231, 2.1927731779171893, 10.563083864358715, 2.001448286999518, 4.804541102597839, 0.608351639401759, 0.70471988284621

In [0]:
# predictions.write.mode("overwrite").parquet(f"{blob_url}/predictions_GBT_undersampled")